In [42]:
spark.stop()

In [43]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')
import os, findspark
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
import pubmed_parser as pp

#sys.path.append('/home/ananth/nsf_data_ingestion/')

from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params


def create_session():
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.\
            appName('test_pubmed_parser').\
            config('spark.driver.memory', '20g').\
            config('spark.network.timeout', '600s').\
            config('spark.driver.maxResultSize', '10g').\
            config('spark.executor.memory', '15g').\
            config('spark.kryoserializer.buffer.max', '1g').\
            config('spark.cores.max', '50').\
            getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.2.2-py3.7.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

    logging.info("Spark Session....Created")
    return spark

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]

spark = create_session()
sc = spark.sparkContext

INFO:root:Creating Spark Session....
INFO:root:Spark Session....Created


In [40]:
def testf(gzip_str):
    return gzip_str[1]

docs = sc.wholeTextFiles('test/test/pubmed21n0001.xml.gz').repartition(1000)
gzip_str = docs.flatMap(testf).collect()
pp.parse_medline_xml(gzip_str)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 1002 tasks (1133.6 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
	at org.apache.spark.scheduler.DAGScheduler

In [25]:
docs = sc.wholeTextFiles('test/test/pubmed21n0001.xml.gz').repartition(1000)
dicts = docs.flatMap(parse_gzip_medline_str)
medline_df = dicts.toDF()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 236 in stage 69.0 failed 4 times, most recent failure: Lost task 236.3 in stage 69.0 (TID 4392, ist-deacuna-n2.syr.edu, executor 27): ExecutorLostFailure (executor 27 exited caused by one of the running tasks) Reason: Container from a bad node: container_1617388819238_0288_01_000037 on host: ist-deacuna-n2.syr.edu. Exit status: 1. Diagnostics: Exception from container-launch.
Container id: container_1617388819238_0288_01_000037
Exit code: 1
Stack trace: ExitCodeException exitCode=1: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:604)
	at org.apache.hadoop.util.Shell.run(Shell.java:507)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:789)
	at org.apache.hadoop.yarn.server.nodemanager.LinuxContainerExecutor.launchContainer(LinuxContainerExecutor.java:399)
	at org.apache.hadoop.yarn.server

In [ ]:
import pandas as pd
panda_df = pd.DataFrame(medline_df.head(5), columns=medline_df.columns)
panda_df

In [19]:
!pwd

/home/eileen/nsf_data_ingestion/nsf_data_ingestion/medline


In [18]:
!ls ~/medline_data/pubmed21n0974.xml.gz

.xml.gz  pubmed21n0973.xml.gz
pubmed21n0120.xml.gz  pubmed21n0547.xml.gz  pubmed21n0974.xml.gz
pubmed21n0121.xml.gz  pubmed21n0548.xml.gz  pubmed21n0975.xml.gz
pubmed21n0122.xml.gz  pubmed21n0549.xml.gz  pubmed21n0976.xml.gz
pubmed21n0123.xml.gz  pubmed21n0550.xml.gz  pubmed21n0977.xml.gz
pubmed21n0124.xml.gz  pubmed21n0551.xml.gz  pubmed21n0978.xml.gz
pubmed21n0125.xml.gz  pubmed21n0552.xml.gz  pubmed21n0979.xml.gz
pubmed21n0126.xml.gz  pubmed21n0553.xml.gz  pubmed21n0980.xml.gz
pubmed21n0127.xml.gz  pubmed21n0554.xml.gz  pubmed21n0981.xml.gz
pubmed21n0128.xml.gz  pubmed21n0555.xml.gz  pubmed21n0982.xml.gz
pubmed21n0129.xml.gz  pubmed21n0556.xml.gz  pubmed21n0983.xml.gz
pubmed21n0130.xml.gz  pubmed21n0557.xml.gz  pubmed21n0984.xml.gz
pubmed21n0131.xml.gz  pubmed21n0558.xml.gz  pubmed21n0985.xml.gz
pubmed21n0132.xml.gz  pubmed21n0559.xml.gz  pubmed21n0986.xml.gz
pubmed21n0133.xml.gz  pubmed21n0560.xml.gz  pubmed21n0987.xml.gz
pubmed21n0134.xml.gz  pubmed21n0561.xml.gz  pubmed21n0988.xm

In [20]:
import pubmed_parser as pp
path_f = "/home/eileen/medline_data/pubmed21n0974.xml.gz"
dict = pp.parse_medline_xml(path_f)
local_df = pd.DataFrame(dict)
local_df

,title,abstract,journal,authors,pubdate,pmid,mesh_terms,publication_types,chemical_list,keywords,doi,references,delete,affiliations,pmc,other_id,medline_ta,nlm_unique_id,issn_linking,country
0,Integrated wastewater management: The future o...,,Integrated environmental assessment and manage...,G Tchobanoglous,2019,30589997,D003247:Conservation of Natural Resources; D01...,D016428:Journal Article,D062065:Waste Water,,10.1002/ieam.4103,,False,Department of Civil and Environmental Engineer...,,,Integr Environ Assess Manag,101234521,1551-3777,United States
1,Learned Discourses: Timely Scientific Opinions.,,Integrated environmental assessment and manage...,,2019,30589998,D012586:Science,D016428:Journal Article,,,10.1002/ieam.4114,,False,,,,Integr Environ Assess Manag,101234521,1551-3777,United States
2,Efficacy and tolerability of adjunctive peramp...,PURPOSE\nThere is limited data on the use of p...,Korean journal of pediatrics,Y Yun;D Kim;YJ Lee;S Kwon;SK Hwang,2019,30590000,,D016428:Journal Article,,Adverse events; Child; Efficacy; Perampanel; T...,10.3345/kjp.2018.06863,21075011;21426333;22738069;23179642;23772853;2...,False,"Department of Pediatrics, School of Medicine, ...",,,Korean J Pediatr,101215374,1738-1061,Korea (South)
3,Clinical practice guideline for the diagnosis ...,The Committee on Pediatric Obesity of the Kore...,Korean journal of pediatrics,DY Yi;SC Kim;JH Lee;EH Lee;JY Kim;YJ Kim;KS Ka...,2019,30589999,,D016428:Journal Article,,Child; Diagnosis; Guideline; Obesity; Treatment,10.3345/kjp.2018.07360,10730683;10805034;11126229;11134471;11291375;1...,False,"Department of Pediatrics, Chung-Ang University...",,,Korean J Pediatr,101215374,1738-1061,Korea (South)
4,The association between preeclampsia and autis...,PURPOSE\nThis meta-analysis pooled relevant ca...,Korean journal of pediatrics,E Jenabi;M Karami;S Khazaei;S Bashirian,2019,30590001,,D016428:Journal Article,,Autism; Autism spectrum disorder; Preeclampsia...,10.3345/kjp.2018.07010,15184241;15870155;17404128;19841112;19936906;2...,False,Pediatric Developmental Disorders Research Cen...,,,Korean J Pediatr,101215374,1738-1061,Korea (South)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,"Azaindolo[3,2,1-jk]carbazoles: New Building Bl...",The preparation and characterization of 12 aza...,"Chemistry (Weinheim an der Bergstrasse, Germany)",T Kader;B Stöger;J Fröhlich;P Kautny,2019,30620787,,D016428:Journal Article,,C−H activation; carbazole; functional organic ...,10.1002/chem.201805578,12746820;15844924;16402846;16551119;17165682;1...,False,"Institute of Applied Synthetic Chemistry, TU W...",,,Chemistry,9513783,0947-6539,Germany
29996,Heteroatom-Doped Porous Carbon Materials with ...,The design of carbon-based materials with a hi...,Angewandte Chemie (International ed. in English),H Jin;X Feng;J Li;M Li;Y Xia;Y Yuan;C Yang;B D...,2019,30620788,,D016428:Journal Article,,heteroatoms; high-density carbon materials; po...,10.1002/anie.201813686,,False,College of Chemistry and Materials Engineering...,,,Angew Chem Int Ed Engl,0370543,1433-7851,Germany
29997,Cervical and High-Thoracic Dorsal Root Ganglio...,INTRODUCTION\nDorsal root ganglion stimulation...,Neuromodulation : journal of the International...,GS Piedade;J Vesper;A Chatzikalfas;PJ Slotty,2019,30620789,D000328:Adult; D000368:Aged; D002574:Cervical ...,D002363:Case Reports,,Dorsal root ganglion stimulation; neuromodulat...,10.1111/ner.12916,,False,"Department of Neurosurgery, Heinrich-Heine-Uni...",,,Neuromodulation,9804159,1094-7159,United States
29998,Erratum.,,Conservation biology : the journal of the Soci...,,2019,30620791,,D016425:Published Erratum,,,10.1111/cobi.13243,,False,,,,Conserv Biol,9882301,0888-8892,United States


In [21]:
pp.__version__

'0.2.2'

In [22]:
!pip list

Package                            Version    Location
---------------------------------- ---------- -------------------------------
absl-py                            0.11.0
affinegap                          1.11
alabaster                          0.7.12
alembic                            1.4.3
anaconda-client                    1.7.2
anaconda-navigator                 1.9.7
anaconda-project                   0.8.3
apache-airflow                     1.10.12
apispec                            1.3.3
argcomplete                        1.12.1
asn1crypto                         0.24.0
astor                              0.8.1
astroid                            2.2.5
astropy                            3.2.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.3.0
attrs                              19.3.0
awscli                             1.18.158
Babel                              2.7.0
backcall                           0.1.

In [23]:
pp.__file__

'/home/eileen/.local/lib/python3.7/site-packages/pubmed_parser/__init__.py'

In [25]:
!ls /home/eileen/.local/lib/python3.7/site-packages/

__init__.py	   pubmed_oa_parser.py	 __pycache__
medline_parser.py  pubmed_web_parser.py  utils.py


In [ ]:
rdd0 = sc.parallelize(list(range(10000))).repartition(1000)

In [ ]:
def get_hostnode(e):
    import sys
    import socket
    
    return str(sys.version_info), str(socket.gethostname())


In [ ]:
def check_version(e):
    import pyarrow as pa
    hdfs = pa.hdfs.connect()
    
    return str(pa.__version__), str(hdfs)

In [ ]:
rdd0.map(check_version).distinct().collect()

In [ ]:
hdfs = pa.hdfs.connect()

In [ ]:
print(hdfs)

In [ ]:
rdd0.map(get_hostnode).distinct().collect()

In [ ]:
get_hostnode(2)

In [ ]:
!/home/tozeng/anaconda3/bin/python --version

In [35]:
re.__version__

'2.2.1'

In [ ]:
test_p = spark.sparkContext.parallelize(file_paths)
test_p.collect()

In [38]:
def find_journal(gzip):
    import re
    l = re.findall('<Title>[\s\S]*?<\/Title>', gzip[1])

    return(l)

In [ ]:
def test_medline_parquet(gzip):
    import pubmed_parser as pa
    dicts = pa.parse_medline_xml(gzip[1])
    return [Row(**doc) for doc in dicts]

In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(find_journal)
dicts.take(10)

In [ ]:

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]



In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(parse_gzip_medline_str)
dicts.take(10)



In [ ]:
!hdfs dfs 

In [ ]:
spark.stop()

In [ ]:
def test_medline_parquet(file_path):
    import subprocess
    (ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
    
    return (out,err)

In [ ]:
def run_cmd(args_list):
        """
        run linux commands
        """
        # import subprocess
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err 

In [ ]:
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
out.decode('ascii').split('\n')

In [ ]:
err = test_p.map(test_medline_parquet).take(1)

In [ ]:
err[0][1].decode('ascii').split('\n')

In [ ]:
tet = spark.sparkContext.wholeTextFiles('test/test/*.xml.gz')

In [ ]:
tet.map(lambda x : x[0]).take(1)

In [ ]:
pa.hdfs.connect()

In [ ]:
spark.stop()